In [36]:
import numpy as np
from pygsti.processors import QubitProcessorSpec as QPS
from pygsti.models import create_cloud_crosstalk_model
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit import transpile
from pygsti.circuits import Circuit
from qiskit import QuantumCircuit
import random

In [ ]:
#ibm brussels
backend = service.backend('ibm_brussels')

# 디바이스 정보 가져오기
n_qubits = backend.configuration().n_qubits
coupling_map = backend.configuration().coupling_map

# 비표준 게이트 유니터리 정의
ECR_unitary = np.array([
    [1, 0, 0, 0],
    [0, 0, 1, 0],
    [0, 1, 0, 0],
    [0, 0, 0, 1]
])
SX_unitary = np.array([
    [0.5 + 0.5j, 0.5 - 0.5j],
    [0.5 - 0.5j, 0.5 + 0.5j]
])
I_unitary = np.array([
    [1,0],
    [0,1]
])
X_unitary = np.array([
    [0,1],
    [1,0]
])
theta = np.pi / 2
RZ_unitary = np.array([
    [np.exp(-1j * theta / 2), 0],
    [0, np.exp(1j * theta / 2)]
])

nonstd_gate_unitaries = {
    'ECR': ECR_unitary,
    'sx': SX_unitary,
    'rz' : RZ_unitary,
    'x' : X_unitary,
    'id' : I_unitary
}

# 서브셋 큐비트 정의 (5개 선택)
# 프로세서 사양 생성
subset_qubits = [0, 1, 2, 3, 4] 
qubit_labels = [f'Q{i}' for i in subset_qubits]
gates = ['ECR', 'id', 'x', 'sx', 'rz']
#원소는 시퀀스의 길이를 의미하고 리스트 크기를 이용하여 회로 갯수 결정
max_length = [1, 2, 4, 8, 16] 
pspec = QPS(
    num_qubits=len(subset_qubits),
    gate_names=gates,
    nonstd_gate_unitaries=nonstd_gate_unitaries,
    qubit_labels=[f'Q{i}' for i in subset_qubits],
    availability = {
    'ECR': [('Q0', 'Q1'), ('Q1', 'Q2'), ('Q2', 'Q3')],
    'id': [('Q0',), ('Q1',), ('Q2',), ('Q3',), ('Q4',)],
    'x': [('Q0',), ('Q1',), ('Q2',), ('Q3',), ('Q4',)],
    'sx': [('Q0',), ('Q1',), ('Q2',), ('Q3',), ('Q4',)],
    'rz': [('Q0',), ('Q1',), ('Q2',), ('Q3',), ('Q4',)]
    }
)
prep_basis = {
    "+X": ("Gx",),
    "-X": ("Gx", "Gx"),
    "+Y": ("Gy",),
    "-Y": ("Gy", "Gy"),
    "+Z": ("Gz",),
    "-Z": ("Gz", "Gz"),
    "+I": ("Gi",),
    "-I": ("Gi", "Gi")
}
meas_basis = {
    "+X": ("Gx",),
    "-X": ("Gx", "Gx"),
    "+Y": ("Gy",),
    "-Y": ("Gy", "Gy"),
    "+Z": ("Gz",),
    "-Z": ("Gz", "Gz"),
    "+I": ("Gi",),
    "-I": ("Gi", "Gi")
}
pauli_basis_dicts = (prep_basis, meas_basis)


In [ ]:
model = create_cloud_crosstalk_model(pspec)

experiment = []
for _ in range(len(pspec.qubit_labels)):
    gate_sequence = []
    for i in range(max_length[i]):
        # Q0에 Identity 게이트 추가
        layer = [('Gi', 'Q0')]  
        for qubit in ['Q1', 'Q2', 'Q3', 'Q4']:
            random_gate = random.choice(['Gx', 'Gy', 'Gz', 'Gi'])
            layer.append((random_gate, qubit))
        gate_sequence.extend(layer)

    circuit = Circuit(gate_sequence)
    experiment.append(circuit)


In [38]:
gate_mapping = {
    'Gi': 'id',
    'Gx': 'x',
    'Gy': 'y',
    'Gz': 'z'
}
def convert_to_qiskit(py_gsti_circuit, num_qubits):
    qiskit_circuit = QuantumCircuit(num_qubits)
    for gate, qubit in py_gsti_circuit:
        qiskit_gate = gate_mapping.get(gate)
        if qiskit_gate:
            getattr(qiskit_circuit, qiskit_gate)(int(qubit[1]))  # Qubit index는 문자열의 숫자 부분 사용
    return qiskit_circuit

qiskit_circuits = [convert_to_qiskit(circuit, len(pspec.qubit_labels)) for circuit in experiment]
transpiled_circuits = [transpile(qc, backend,optimization_level=0) for qc in qiskit_circuits]

job = backend.run(transpiled_circuits,shots=1024)
print(f"Job ID: {job.job_id()}")

# 결과 가져오기
result = job.result()
print("Experiment completed. Results:")
for idx, counts in enumerate(result.get_counts()):
    print(f"Circuit {idx} Results: {counts}")


C:\Users\wjdrm\AppData\Local\Temp\ipykernel_24016\1257324899.py:18: DeprecationWarning: backend.run() and related sessions methods are deprecated  as of qiskit-ibm-runtime 0.23 and will be removed no sooner than 6 months after the release date. More details can be found in the primitives migration guide https://docs.quantum.ibm.com/migration-guides/qiskit-runtime.
  job = backend.run(transpiled_circuits,shots=1024)


Job ID: cx2n3zzpjw30008yykag


KeyboardInterrupt: 